<a href="https://colab.research.google.com/github/manjunathsirur1994/Time_series_forecasting/blob/main/Regression_for_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import math

In [75]:
ticker = 'LLOYDSME.NS'
df = yf.download(ticker, start='2020-01-01', multi_level_index=False)
df

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2023-07-17 00:00:00+00:00,562.558289,563.299988,582.700012,550.500000,554.000000,417232
2023-07-18 00:00:00+00:00,543.583313,544.299988,573.900024,531.000000,573.900024,284708
2023-07-19 00:00:00+00:00,565.504395,566.250000,572.000000,541.250000,564.700012,208966
2023-07-20 00:00:00+00:00,563.307312,564.049988,572.700012,557.000000,572.700012,108284
2023-07-21 00:00:00+00:00,557.964355,558.700012,564.500000,550.049988,564.500000,189224
...,...,...,...,...,...,...
2024-11-05 00:00:00+00:00,975.500000,975.500000,987.599976,966.950012,970.250000,156433
2024-11-06 00:00:00+00:00,1016.049988,1016.049988,1020.000000,971.150024,971.150024,429220
2024-11-07 00:00:00+00:00,995.750000,995.750000,1030.000000,990.000000,1014.500000,178607


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 326 entries, 2023-07-17 00:00:00+00:00 to 2024-11-11 00:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  326 non-null    float64
 1   Close      326 non-null    float64
 2   High       326 non-null    float64
 3   Low        326 non-null    float64
 4   Open       326 non-null    float64
 5   Volume     326 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 17.8 KB


In [77]:
df = df[['Close']]
df.head()
print(df.shape)

(326, 1)


In [78]:
df = df.resample('B').last()
df.fillna(method='bfill')
df.shape

<ipython-input-78-fda4561ce666>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill')


(346, 1)

In [79]:
df.reset_index(inplace=True)
df

,Date,Close
0,2023-07-17 00:00:00+00:00,563.299988
1,2023-07-18 00:00:00+00:00,544.299988
2,2023-07-19 00:00:00+00:00,566.250000
3,2023-07-20 00:00:00+00:00,564.049988
4,2023-07-21 00:00:00+00:00,558.700012
...,...,...
341,2024-11-05 00:00:00+00:00,975.500000
342,2024-11-06 00:00:00+00:00,1016.049988
343,2024-11-07 00:00:00+00:00,995.750000
344,2024-11-08 00:00:00+00:00,1006.049988


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype              
---  ------  --------------  -----              
 0   Date    346 non-null    datetime64[ns, UTC]
 1   Close   326 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 5.5 KB


In [81]:
# df['Close_diff'] = df['Close'].diff()
# df['Close_diff_2'] = df['Close'].diff(2)
# df['Close_diff_3'] = df['Close'].diff(3)
df['Close_lag1'] = df['Close'].shift(1)
df['Close_lag2'] = df['Close'].shift(2)
df['Close_lag3'] = df['Close'].shift(3)
df['Close_lag4'] = df['Close'].shift(4)
df['Close_lag5'] = df['Close'].shift(5)
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['dayofweek'] = df['Date'].dt.dayofweek
df['day_name'] = df['Date'].dt.day_name()
df['quarter'] = df['Date'].dt.quarter
df['dayofyear'] = df['Date'].dt.dayofyear
df['weekofyear'] = df['Date'].dt.isocalendar().week
df['isweekend'] = df['dayofweek'] >= 5
df = df.dropna()
df = df.drop(columns=['Date'])
df

,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,day_name,quarter,dayofyear,weekofyear,isweekend
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,Monday,3,205,30,False
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,Tuesday,3,206,30,False
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,Wednesday,3,207,30,False
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,Thursday,3,208,30,False
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,Friday,3,209,30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,Tuesday,4,310,45,False
342,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,Wednesday,4,311,45,False
343,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,Thursday,4,312,45,False
344,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,Friday,4,313,45,False


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227 entries, 5 to 345
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Close       227 non-null    float64
 1   Close_lag1  227 non-null    float64
 2   Close_lag2  227 non-null    float64
 3   Close_lag3  227 non-null    float64
 4   Close_lag4  227 non-null    float64
 5   Close_lag5  227 non-null    float64
 6   year        227 non-null    int32  
 7   month       227 non-null    int32  
 8   day         227 non-null    int32  
 9   dayofweek   227 non-null    int32  
 10  day_name    227 non-null    object 
 11  quarter     227 non-null    int32  
 12  dayofyear   227 non-null    int32  
 13  weekofyear  227 non-null    UInt32 
 14  isweekend   227 non-null    bool   
dtypes: UInt32(1), bool(1), float64(6), int32(6), object(1)
memory usage: 20.8+ KB


In [83]:
dummies = pd.get_dummies(df['day_name'])
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns=['day_name'])
df

,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,quarter,dayofyear,weekofyear,isweekend,Friday,Monday,Thursday,Tuesday,Wednesday
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,3,205,30,False,False,True,False,False,False
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,3,206,30,False,False,False,False,True,False
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,3,207,30,False,False,False,False,False,True
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,3,208,30,False,False,False,True,False,False
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,3,209,30,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,4,310,45,False,False,False,False,True,False
342,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,4,311,45,False,False,False,False,False,True
343,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,4,312,45,False,False,False,True,False,False
344,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,4,313,45,False,True,False,False,False,False


In [84]:
df.replace({True: 1, False: 0}, inplace=True)
df

<ipython-input-84-55e9c1cfbffd>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({True: 1, False: 0}, inplace=True)


,Close,Close_lag1,Close_lag2,Close_lag3,Close_lag4,Close_lag5,year,month,day,dayofweek,quarter,dayofyear,weekofyear,isweekend,Friday,Monday,Thursday,Tuesday,Wednesday
5,575.099976,558.700012,564.049988,566.250000,544.299988,563.299988,2023,7,24,0,3,205,30,0,0,1,0,0,0
6,574.900024,575.099976,558.700012,564.049988,566.250000,544.299988,2023,7,25,1,3,206,30,0,0,0,0,1,0
7,574.500000,574.900024,575.099976,558.700012,564.049988,566.250000,2023,7,26,2,3,207,30,0,0,0,0,0,1
8,575.500000,574.500000,574.900024,575.099976,558.700012,564.049988,2023,7,27,3,3,208,30,0,0,0,1,0,0
9,596.900024,575.500000,574.500000,574.900024,575.099976,558.700012,2023,7,28,4,3,209,30,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,975.500000,970.150024,978.099976,972.849976,955.750000,950.849976,2024,11,5,1,4,310,45,0,0,0,0,1,0
342,1016.049988,975.500000,970.150024,978.099976,972.849976,955.750000,2024,11,6,2,4,311,45,0,0,0,0,0,1
343,995.750000,1016.049988,975.500000,970.150024,978.099976,972.849976,2024,11,7,3,4,312,45,0,0,0,1,0,0
344,1006.049988,995.750000,1016.049988,975.500000,970.150024,978.099976,2024,11,8,4,4,313,45,0,1,0,0,0,0


In [85]:
x = df.drop(columns=['Close'])
y = df['Close']

In [86]:
df.shape

(227, 19)

In [87]:
x_train = x.head(226)
y_train = y.head(226)
x_test = x.tail(1)
y_test = y.tail(1)

In [88]:
y_train.tail(1)

,Close
344,1006.049988


In [89]:
y_test

,Close
345,992.0


In [90]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [91]:
lr.score(x_train, y_train)

0.9868439979925546

In [92]:
y_pred = lr.predict(x_test)
y_pred

array([1015.64900474])

In [93]:
y_test - y_pred

,Close
345,-23.649005


In [94]:
y_test

,Close
345,992.0


In [95]:
mean_squared_error(y_test, y_pred)

559.2754252659412

In [96]:
math.sqrt(mean_squared_error(y_test, y_pred))

23.649004741551835

In [97]:
y_train.tail(1)

,Close
344,1006.049988


In [98]:
y_test

,Close
345,992.0


In [99]:
lr.coef_

array([ 1.00648373e+00, -1.71232944e-01,  2.21914355e-01, -6.90813480e-02,
       -2.11919815e-03,  7.24641488e+00,  2.57196992e+00, -1.67684377e-02,
        5.61106720e-01, -2.52009585e+00, -1.65135136e-01,  9.31453289e-01,
       -1.66533454e-15,  3.55781800e-02,  1.11392007e+00,  1.74826252e+00,
       -9.69527982e-01, -1.92823279e+00])